In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections
import pickle
import gzip
import re

from tf.app import use

In [3]:
A = use("CLARIAH/descartes-tf:clone", checkout="clone", hoist=globals())

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
volume,8,85241.88,100
letter,725,940.60,100
page,2884,236.45,100
postscriptum,56,46.79,0
opener,545,1.97,0
closer,541,13.10,1
address,86,15.22,0
head,725,23.37,2
p,8438,80.82,100
sentence,14332,45.74,96


# Parallels

We make edges between similar sentences.

When are sentences similar?

If a certain distance metric is above a certain threshold.

We choose this metric:

* we reduce a sentence to the set of words in it, excluding punctuation.
* the similarity between two sentences is the size of the intersection divided by the size of the union of their sets times 100.

# Preparation

We pre-compute trimmed contents for all sentences in the base text.
But we weed out the sentences that do not start with a capital letter.

In [12]:
WHITE_RE = re.compile(r"\s\s+")

def makeSent(sentence):
    text = T.text(sentence).strip()
    text = WHITE_RE.sub(" ", text)
    
    if not text:
        return ""
    firstLetter = text[0]
    if not firstLetter.isalpha() or firstLetter.upper() != firstLetter:
        return ""
    return text

Weed out the sentences that do not start with a capital letter.

In [13]:
sentences = {}

for sentence in F.otype.s("sentence"):
    sent = makeSent(sentence)
    if sent:
        sentences[sentence] = sent

nSentences = len(sentences)
print(f"{nSentences} sentences")

12690 sentences


# Measure

`pip install python-Levenshtein`

In [14]:
from Levenshtein import ratio

In [15]:
def sim(lSent, mSent):
    return 100 * ratio(lSent, mSent)

# Compute all similarities

We are going to perform several millions of comparisons, each of which is more than an elemetary operation.

Let's measure time.

In [16]:
THRESHOLD = 80


def computeSim(limit=None):
    similarity = {}

    sentenceNodes = sorted(sentences.keys())
    nSentences = len(sentenceNodes)

    nComparisons = nSentences * (nSentences - 1) // 2

    print(f"{nComparisons} comparisons to make")
    chunkSize = nComparisons // 100

    co = 0
    b = 0
    si = 0
    p = 0

    A.indent(reset=True)

    stop = False
    for i in range(nSentences):
        nodeI = sentenceNodes[i]
        sentenceI = sentences[nodeI]
        for j in range(i + 1, nSentences):
            nodeJ = sentenceNodes[j]
            sentenceJ = sentences[nodeJ]
            s = sim(sentenceI, sentenceJ)
            co += 1
            b += 1
            if b == chunkSize:
                p += 1
                A.info(f"{p:>3}% - {co:>12} comparisons and {si:>10} similarities")
                b = 0
                if limit is not None and p >= limit:
                    stop = True
                    break

            if s < THRESHOLD:
                continue
            similarity[(nodeI, nodeJ)] = sim(sentenceI, sentenceJ)
            si += 1
        if stop:
            break

    A.info(f"{p:>3}% - {co:>12} comparisons and {si:>10} similarities")
    return similarity

We are going to run it to a few % first and do some checks then.

In [17]:
similarity = computeSim(limit=10)

80511705 comparisons to make
  1.46s   1% -       805117 comparisons and          1 similarities
  3.15s   2% -      1610234 comparisons and          1 similarities
  5.11s   3% -      2415351 comparisons and          1 similarities
  7.51s   4% -      3220468 comparisons and          3 similarities
  9.78s   5% -      4025585 comparisons and          3 similarities
    12s   6% -      4830702 comparisons and          3 similarities
    14s   7% -      5635819 comparisons and          3 similarities
    17s   8% -      6440936 comparisons and          3 similarities
    18s   9% -      7246053 comparisons and          3 similarities
    21s  10% -      8051170 comparisons and          8 similarities
    21s  10% -      8051170 comparisons and          8 similarities


We check the sanity of the results.

In [18]:
print(min(similarity.values()) if len(similarity) else 0)
print(max(similarity.values()) if len(similarity) else 0)

81.13207547169812
100.0


In [19]:
eq = [x for x in similarity.items() if x[1] >= 100]
neq = [x for x in similarity.items() if x[1] <= THRESHOLD]

In [20]:
print(len(eq))
print(len(neq))

1
0


In [21]:
print(eq[0] if len(eq) else 0)
print(neq[0] if len(neq) else 0)

((708472, 716876), 100.0)
0


Looks good.

Now the whole computation.

But if we have done this before, and nothing has changed, we load previous results from disk.

If we do not find previous results, we compute them and save the results to disk.

In [22]:
PARA_DIR = f"{A.tempDir}/parallels"


def writeResults(data, location, name):
    if not os.path.exists(location):
        os.makedirs(location, exist_ok=True)
    path = f"{location}/{name}"
    with gzip.open(path, "wb") as f:
        pickle.dump(data, f)
    print(f"Data written to {path}")


def readResults(location, name):
    path = f"{location}/{name}"
    if not os.path.exists(path):
        print(f"File not found: {path}")
        return None
    with gzip.open(path, "rb") as f:
        data = pickle.load(f)
    print(f"Data read from {path}")
    return data

In [23]:
similarity = readResults(PARA_DIR, f"sim-{A.version}.zip")
if not similarity:
    similarity = computeSim()
    writeResults(similarity, PARA_DIR, f"sim-{A.version}.zip")

File not found: /Users/me/github/CLARIAH/descartes-tf/_temp/parallels/sim-1.0.zip
80511705 comparisons to make
  1.44s   1% -       805117 comparisons and          1 similarities
  3.12s   2% -      1610234 comparisons and          1 similarities
  5.07s   3% -      2415351 comparisons and          1 similarities
  7.45s   4% -      3220468 comparisons and          3 similarities
  9.70s   5% -      4025585 comparisons and          3 similarities
    12s   6% -      4830702 comparisons and          3 similarities
    14s   7% -      5635819 comparisons and          3 similarities
    16s   8% -      6440936 comparisons and          3 similarities
    18s   9% -      7246053 comparisons and          3 similarities
    20s  10% -      8051170 comparisons and          8 similarities
    22s  11% -      8856287 comparisons and          8 similarities
    24s  12% -      9661404 comparisons and          9 similarities
    26s  13% -     10466521 comparisons and          9 similarities
    2

In [24]:
len(similarity)

1199

So, not too many similarities.

Let's find out which sentences have the most correspondences.

In [25]:
parallels = {}

for (sentence, m) in similarity:
    parallels.setdefault(sentence, set()).add(m)
    parallels.setdefault(m, set()).add(sentence)

print(f"{len(parallels)} out of {nSentences} sentences have at least one similar line")

354 out of 12690 sentences have at least one similar line


In [26]:
rankedParallels = sorted(
    parallels.items(),
    key=lambda x: (-len(x[1]), x[0]),
)

In [27]:
seen = set()


def getPos(node):
    sec = A.sectionStrFromNode(node)
    return f"{sec:<15} @ {node:>5}"


for (sentence, paras) in rankedParallels:
    if sentence in seen:
        continue
    plural = " " if len(paras) == 1 else "s"
    prefix = f"{len(paras):>4} sibling{plural} of "
    blank = " " * len(prefix)
    print(f"{prefix}{getPos(sentence)} ====== {T.text(sentence).strip()}")
    for para in paras:
        sim = similarity[(sentence, para)] if (sentence, para) in similarity else similarity[(para, sentence)]
        print(f"{blank}{getPos(para)} ={sim:>3}%= {T.text(para).strip()}")
        seen.add(para)
    print("")
    seen.add(sentence)

  28 siblings of 3 3216:9        @ 714281 ====== Adresse:
                 8 8263bis:9     @ 722705 =100.0%= Adresse:
                 6 6390:8        @ 717845 =100.0%= Adresse:
                 5 5364:11       @ 717464 =100.0%= Adresse:
                 4 4245:46       @ 715036 =100.0%= Adresse:
                 4 4246:9        @ 715040 =100.0%= Adresse:
                 7 7585:8        @ 720676 =100.0%= Adresse:
                 7 7537:13       @ 719660 =100.0%= Adresse:
                 5 5373:10       @ 717614 =100.0%= Adresse:
                 4 4257:9        @ 715312 =100.0%= Adresse:
                 7 7529:19       @ 719540 =100.0%= Adresse:
                 7 7586:14       @ 720692 =100.0%= Adresse:
                 6 6493:12       @ 719038 =100.0%= Adresse:
                 6 6394:9        @ 717889 =100.0%= Adresse:
                 7 7560:17       @ 720327 =100.0%= Adresse:
                 7 7596:10       @ 720840 =100.0%= Adresse:
                 6 6395:12       @ 717897 

Additional questions as an exercise:

* how many sentences have just one correspondence?
  (look at the tail of rankedParallels)
* make an overview of exactly how wide-spread parallel lines are.

# Add parallels to the TF dataset

We can add this information to the corpus dataset as an *edge feature*.

An edge feature links two nodes and may annotate that link with a value.

For parallels, we link each line to each of its parallel lines and we annotate that link with the similarity between
the two lines. The similarity is a percentage, and we round it to integer values.

If *n1* is similar to *n2*, then *n2* is similar to *n1*.
In order to save space, we only add such links once.

We can then use
[`E.sim.b(node)`](https://annotation.github.io/text-fabric/Api/Features/#edge-features)
to find all nodes that are parallel to node.


In [28]:
from tfFromTei import SETTINGS

metaData = {
    "": SETTINGS["generic"],
    "sim": {
        "valueType": "int",
        "edgeValues": True,
        "description": (
            "similarity between sentences "
            " based on the Levenshtein ratio"
        ),
    },
}

In [29]:
simData = {}
for ((f, t), d) in similarity.items():
    simData.setdefault(f, {})[t] = int(round(d))

In [30]:
backendBase = os.path.expanduser(f"~/{A.backend}")
mod = "parallels"
path = f"{A.context.org}/{A.context.repo}/{mod}/tf"
location = f"{backendBase}/{path}"
module = A.version

In [31]:
TF.save(
    edgeFeatures=dict(sim=simData), metaData=metaData, location=location, module=module, silent="auto"
)

  0.00s Exporting 0 node and 1 edge and 0 config features to /Users/me/github/CLARIAH/descartes-tf/parallels/tf/1.0:
   |     0.00s T sim                  to /Users/me/github/CLARIAH/descartes-tf/parallels/tf/1.0
  0.00s Exported 0 node features and 1 edge features and 0 config features to /Users/me/github/CLARIAH/descartes-tf/parallels/tf/1.0


True